In [1]:
import time
from lxml import html
import regex

import requests
from datetime import datetime
import pandas
import argparse

In [ ]:
parser = argparse.ArgumentParser(
                    prog='get_linkedin_jobposts',
                    description='Scrapes LinkedIn job posts for a given city'
                    )

parser.add_argument('-l', '--location', default='vancouver')

args, unknown = parser.parse_known_args()

# global arguments
location = args.location
today = datetime.now().strftime('%Y%m%d')

In [ ]:
# TODO:
# - break up functions
# - back out search terms (from log file)
# - resume search
# - add argparse for more locations

# src = '/home/qcx201/Projects/jobs/linkedin/log/get_jobposts-vancouver.log'
# with open(src, 'r') as f:
#     log = f.read()

# lines = log.split('\n')
# lines = [line for line in lines if 'search-keyword' in line]
# keywords = [ln.split(': ')[-1] for ln in lines]
# keywords

In [ ]:
def search_jobs(params):
    """
    Generator function to scrape job posts from LinkedIn.
    Note: Linkedin limits outputs to 1000 results per search parameter.
    """
    
    # URL for LinkedIn job posts
    # Note: This URL may change, and scraping LinkedIn may violate their terms of service.
    # Use responsibly and consider using their official API if available.
    url = 'https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search'
    headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/136.0.0.0 Safari/537.36'
        }
    
    # request the job postings
    sleep = 1
    while True:
        resp = requests.get(url, params=params, headers=headers)
        if resp.status_code == 429: # too many requests
            time.sleep(sleep)
            sleep += 1
        else:
            break
        

    print('resp-url:', resp.url)
    
    # exit function if bad response or empty HTML document
    if (not resp) or ('<!DOCTYPE html>\n\n<!---->' in resp.text):
        print('-'*50)
        print(f'request-url [status {resp.status_code}]: {resp.url}')
        print('Bad request or no more jobs found.') # TODO: 
        yield

    # iterate over job list items
    doc = html.fromstring(resp.content)
    lis = doc.xpath('//li')
    
    for li in lis:

        # get the text content of the job posting
        texts = li.xpath('.//text()')
        texts = [regex.sub(r'\s+', ' ', x).strip() for x in texts]
        
        # remove empty strings
        texts = [x for x in texts if x] 

        # remove duplicate text
        texts = list(dict.fromkeys(texts))

        # get links of job postings
        anchors = li.xpath('.//a')
        hrefs = [a.get('href') for a in anchors]

        # yield result
        res = {
            'position' : texts[0]   if len(texts) > 0 else None,
            'company' : texts[1]    if len(texts) > 1 else None,
            'location' : texts[2]   if len(texts) > 2 else None,
            'status' : texts[3]     if len(texts) > 3 else None,
            'job_url' : hrefs[0].split('?')[0],
            'firm_url': hrefs[1]    if len(hrefs) > 1 else None,
            'keywords' : params['keywords'],
        }

        yield res

In [18]:
def get_number_jobs(location):
    '''
    Get number of jobs from search page
    '''
    
    # approx. number of job postings
    url = f'https://www.linkedin.com/jobs/search?location={location}'
    resp = requests.get(url)
    doc = html.fromstring(resp.content)
    njobs = doc.xpath('//span[@class="results-context-header__job-count"]//text()')[0]

    return int(''.join(c for c in njobs if c.isnumeric()))

In [ ]:
def save_data(data, label=today):
    '''
    Save data
    '''

    df = pandas.DataFrame(data)
    
    loc = ''.join(c for c in location if c.isalpha())
    
    dst = f'../data/jobposts/{label}-{loc}.csv'
    df.to_csv(dst, index=False)

    print(f'Saved {df.shape}:', dst)
    print('Now:', datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

In [ ]:
def get_freq(data):
    '''
    Get keyword frequency from job post and companies
    
    TODO: create filter for searched keywords
    '''
    
    df = pandas.DataFrame(data)

    # get keywords from job postings
    pos_str = ' '.join(df['position'].tolist())
    comp_str = ' '.join(df['company'].tolist())
    join_str = pos_str + ' ' + comp_str
    join_str = join_str.lower()

    # remove non-alphabetic characters
    remove = [c for c in set(join_str) if not c.isalpha()]
    for c in remove:
        join_str = join_str.replace(c, ' ')

    words = join_str.split()

    # get word frequencies
    res = [{'keyword' : w, 'count' : words.count(w)} for w in set(words) if len(w) > 2]
    return res

    

In [ ]:
# initialize
if __name__ == '__main__':
    
    print('Location:', location)
    print('Start time:', datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

    print('='*50)

    njobs = get_number_jobs(location)

    # initialize data structures
    
    data = []
    job_urls = set()
    searched_kws = set()
    kw = ''

    past = datetime.now()


Location: vancouver
Start time: 2025-06-23 01:13:06


In [ ]:
if __name__ == '__main__':
    
    # break outer loop if most jobs found or no new keywords found
    while True:

        # break inner loop if no more results in page (usually 10 results)
        run_search, start = True, 0
        while run_search:

            params = {
                'location': location,
                'start': start,
                'keywords' : kw,
                }
            
            search_results = search_jobs(params)
            for search_res in search_results:

                start += 1 # local counter (inner loop)

                # break inner loop if no more results
                if not search_res:
                    run_search = False
                    break

                # add to data if new job url
                job_url = search_res['job_url'].split('?')[0]
                
                if job_url not in job_urls:
                    job_urls.add(job_url)
                    data.append(search_res)

                    # get yield time
                    print(f'[job {len(data):,} of {njobs:,}+]', datetime.now(), job_url)

        # save data
        save_data(data)

        # break outer loop if most jobs found
        if len(data) >= njobs:
            print('Most jobs found.')
            break

        # get word frequencies
        freq = get_freq(data)

        # filter top unsearched keyword
        wf = pandas.DataFrame(freq)
        wf = wf.sort_values(by='count', ascending=False)
        cond = ~wf['keyword'].isin(searched_kws)
        kws = wf[cond]

        # break outer loop if no new keywords found
        if kws.empty:
            print('No more keywords found.')
            break

        # add new keyword to list of searched
        kw = kws.iloc[0]['keyword']
        searched_kws.add(kw)

        print('*'*50)
        print(f'search-keyword ({len(searched_kws):,}):', kw)

    print('=' * 50)
    print('Script complete.')

[job 5,424 of 21,000+] https://ca.linkedin.com/jobs/view/future-opportunities-at-salus-4229947212
[job 5,425 of 21,000+] https://ca.linkedin.com/jobs/view/red-bull-student-marketeer-kpu-at-red-bull-3691569578
[job 5,426 of 21,000+] https://ca.linkedin.com/jobs/view/community-marketing-associate-at-afriex-technologies-4248348098
[job 5,427 of 21,000+] https://ca.linkedin.com/jobs/view/general-manager-lower-mainland-at-go-auto-4176321618
[job 5,428 of 21,000+] https://ca.linkedin.com/jobs/view/graphic-designer-multimedia-at-vibe-lounge-4147779429
[job 5,429 of 21,000+] https://ca.linkedin.com/jobs/view/account-executive-at-sign-in-enterprise-4143830267
[job 5,430 of 21,000+] https://ca.linkedin.com/jobs/view/shift-supervisor-store%23-04924-times-square-robson-denman-at-starbucks-4212298675
[job 5,431 of 21,000+] https://ca.linkedin.com/jobs/view/barista-store%23-68994-pickup-vancouver-convention-centre-at-starbucks-4197732810
[job 5,432 of 21,000+] https://ca.linkedin.com/jobs/view/accou